#**Geocoding Existing Mobile Home Parks (MHP) in Los Angeles County**

This notebook goes through the process of geocoding the mobile home park dataset from LA County. Since our data only included addresses, we will need to convert the addresses to geographic coordinates.

**Import Libraries**

In [1]:
import osmnx as ox
import geopandas as gpd
import contextily as ctx
import plotly.express as px
import pandas as pd

In [2]:
mhp = pd.read_csv( 'Data/CAHCD_1026.csv')

**Data Exploration**

In [3]:
mhp.head()

,Park Name,County,Park Identifier,Park Address,City,Zip,Phone,Fire Authority,Jurisdiction,MH Spaces,RV Lots W/Drains,RV Lots W/O Drains,Operated by
0,IMPERIAL TRAILER PARK,LOS ANGELES,13-0018-MP,"600 N IMPERIAL AVE, HAWTHORNE, CA 90250",HAWTHORNE,90250,NaN,"COUNTY OF LOS ANGELES, 5823 RICKENBACKER RD., ...","IMPERIAL COUNTY BUILDING DEPARTMENT, 801 MAIN ...",8,0,0,"HOLCOM, BRADLEY, PO BOX 3162, SAN DIEGO, CA 92163"
1,CHETS TRAILER PARK,LOS ANGELES,19-0001-MP,"4630 W ROSECRANS AVE , LAWNDALE, CA 90260",LAWNDALE,90260,(310) 676-0663,"COUNTY OF LOS ANGELES, 5823 RICKENBACKER RD., ...","HCD - SOUTHERN AREA OFFICE, 3737 MAIN ST 400, ...",49,0,0,"DOSHI, NAVINCHANDRA, 6418 SPRING PARK AVE, LOS..."
2,A & A MOBILE HOME PARK,LOS ANGELES,19-0003-MP,"7722 ATLANTIC BLVD, CUDAHY, CA 90201",CUDAHY,90201,(310) 676-0663,"COUNTY OF LOS ANGELES, 5823 RICKENBACKER RD., ...","HCD - SOUTHERN AREA OFFICE, 3737 MAIN ST 400, ...",21,0,0,"A & A MHP LLC, 26895 ALISO CREEK RD #B876, ALI..."
3,ACE TRAILER PARK,LOS ANGELES,19-0006-MP,"6508 FLORENCE PL, BELL GARDENS, CA 90201",BELL GARDENS,90201,(213) 927-2868,"COUNTY OF LOS ANGELES, 5823 RICKENBACKER RD., ...","HCD - SOUTHERN AREA OFFICE, 3737 MAIN ST 400, ...",8,0,0,"LUELLA PERDEW REVOCABLE TRUST, PO BOX 3259, WI..."
4,ACORN TP,LOS ANGELES,19-0009-MP,"2818 DURFEE AVE, EL MONTE, CA 91732",EL MONTE,91732,(818) 350-5235,"COUNTY OF LOS ANGELES, 5823 RICKENBACKER RD., ...","HCD - SOUTHERN AREA OFFICE, 3737 MAIN ST 400, ...",20,0,0,"PATTERSON, GLEN, PO BOX 1811, MYRTLE CREEK, OR..."


**Trim the Data**

In [4]:
mhp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596 entries, 0 to 595
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Park Name           596 non-null    object
 1   County              596 non-null    object
 2   Park Identifier     596 non-null    object
 3   Park Address        596 non-null    object
 4   City                596 non-null    object
 5   Zip                 596 non-null    object
 6   Phone               586 non-null    object
 7   Fire Authority      596 non-null    object
 8   Jurisdiction        596 non-null    object
 9   MH Spaces           596 non-null    int64 
 10  RV Lots W/Drains    596 non-null    int64 
 11  RV Lots W/O Drains  596 non-null    int64 
 12  Operated by         595 non-null    object
dtypes: int64(3), object(10)
memory usage: 60.7+ KB


*Because the dataset doesn't have accessive columns, I will keep them in case I want to explore with them later for another analysis